In [2]:
# definition for a binary tree node.
from typing import Optional, List

class TreeNode:
	def __init__(self, val=0, left=None, right=None):
		self.val = val
		self.left = left
		self.right = right

# construct binary tree from list.
def constructBinaryTree(nodes: List[int]) -> TreeNode:
	# if the list is empty, return None.
	if not nodes: return None

	root = TreeNode(nodes[0], None, None)
	queue = [root]

	for i, v in enumerate(nodes[1:]):
		# if v is None, the node is None.
		node = TreeNode(v, None, None) if v else None

		# skip None node, it only a placehoder.
		while queue[0] == None:
			queue.pop(0)
		
		if not i % 2:
			queue[0].left = node
		else:
			queue[0].right = node
			queue.pop(0)
		
		queue.append(node)
	
	return root

def convertBinaryTree2List(root: TreeNode) -> List[int]:
	if not root: return []

	queue = [root]
	ans = []

	while queue:
		n = len(queue)
		for _ in range(n):
			node = queue.pop(0)
			if node:
				queue.append(node.left)
				queue.append(node.right)
				ans.append(node.val)
			else:
				ans.append(node)
	
	for i in range(len(ans)-1, -1, -1):
		if ans[i]: return ans[:i+1]

root = [3,9,20,None,None,15,7]
root_node = constructBinaryTree(root)
tree_list = convertBinaryTree2List(root_node)
tree_list

[3, 9, 20, None, None, 15, 7]

In [3]:
class Solution:
	# Breadth-First Search, also called Level-Order Traversal.
	def levelOrderTraversal(self, root: Optional[TreeNode]) -> List[List[int]]:
		# if the tree is empty, return empty list.
		if not root: return []

		queue = [root]
		ans = []

		while queue:
			# record the node number of the current level.
			n = len(queue)
			cur_level_vals = []
			# traverse the current level nodes.
			for _ in range(n):
				node = queue.pop(0)
				if node.left: queue.append(node.left)
				if node.right: queue.append(node.right)
				cur_level_vals.append(node.val)
			ans.append(cur_level_vals)
		
		return ans
	
	# Depth-First Search, including Pre-Order, In-Order and Post-Order Traversal.
	def preOrderTraversal(self, root: Optional[TreeNode]) -> List[int]:
		# if the tree is empty, return empty list.
		if not root: return []

		ans = [root.val]
		ans += self.preOrderTraversal(root.left)
		ans += self.preOrderTraversal(root.right)

		return ans
	
	def inOrderTraversal(self, root: Optional[TreeNode]) -> List[int]:
		# if the tree is empty, return empty list.
		if not root: return []
		
		ans = []
		ans += self.inOrderTraversal(root.left)
		ans += [root.val]
		ans += self.inOrderTraversal(root.right)

		return ans

	def postOrderTraversal(self, root: Optional[TreeNode]) -> List[int]:
		# if the tree is empty, return empty list.
		if not root: return []

		ans = []
		ans += self.postOrderTraversal(root.left)
		ans += self.postOrderTraversal(root.right)
		ans += [root.val]

		return ans

root = [3,9,20,None,None,15,7]
root = constructBinaryTree(root)

solution = Solution()
solution.levelOrderTraversal(root)
# [[3], [9, 20], [15, 7]]
solution.preOrderTraversal(root)
# [3, 9, 20, 15, 7]
solution.inOrderTraversal(root)
# [9, 3, 15, 20, 7]
solution.postOrderTraversal(root)
# [9, 15, 7, 20, 3]

[9, 15, 7, 20, 3]

In [4]:
class Solution:
	def numTrees(self, n: int) -> int:
		dp = [0] * (n+1)
		# boundary conditions.
		dp[0] = 1
		dp[1] = 1

		for i in range(2, n+1):
			for j in range(i):
				dp[i] += dp[j] * dp[i-j-1]
		
		return dp[n]

	def generateTrees(self, n: int) -> List[Optional[TreeNode]]:
		if not n: return []

		hashtable = dict()


		def _generateTrees(start, end):
			if start > end: return [None]
			if (start, end) in hashtable:
				return hashtable[(start, end)]
			
			ans = []
			
			for i in range(start, end+1):
				left_subtrees = _generateTrees(start, i-1)
				right_subtrees = _generateTrees(i+1, end)
				for left_root in left_subtrees:
					for right_root in right_subtrees:
						root = TreeNode(i, left_root, right_root)
						ans.append(root)
			hashtable[(start, end)] = ans

			return ans


		return _generateTrees(1, n)
	
	def isValidBST(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		def _checkBST(node, lower, upper):
			# leaf node
			if not node: return True
			
			if node.val <= lower or node.val >= upper: return False

			return _checkBST(node.left, lower, node.val) and _checkBST(node.right, node.val, upper)

		return _checkBST(root.left, -2**32-1, root.val) and _checkBST(root.right, root.val, 2**31)
	
	def isValidBST(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		pre_val = root.val
		
		def _inOrderTraversal(root: Optional[TreeNode]):
			if not root: return True

			if not _inOrderTraversal(root.left): return False

			if root.val <= pre_val:
				return False
			
			return _inOrderTraversal(root.right)
		return _inOrderTraversal(root)
	
	def isValidBST3(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		# -inf boundary -2**31-1
		lower_node = TreeNode(-2**31-1, None, None)
		upper_node = TreeNode(2**31, None, None)
		stack = [lower_node, root, upper_node]

		while stack:
			upper_node = stack.pop()
			node = stack.pop()

			while not node.left and not node.right and stack:
				upper_node = node
				node = stack.pop()
			
			if not stack: return True

			if node.left:
				if (node.left.val >= node.val or node.left.val <= stack[-1].val):
					return False
				else:
					stack.append(node.left)
					node.left = None
			
			right_node, node.right = node.right, None
			
			stack.append(node)
			
			if right_node:
				if (right_node.val <= node.val or right_node.val >= upper_node.val):
					return False
				else:
					stack.append(right_node)
			stack.append(upper_node)
	
	def recoverTree(self, root: Optional[TreeNode]) -> None:
		if not root: return root

		self.pre_node = TreeNode(-2**31-1)
		self.swap_node_x, self.swap_node_y = None, None

		def _inOrderTraversal(root):
			if not root:
				print(root)
				return;

			_inOrderTraversal(root.left)

			if root.val <= self.pre_node.val:
				if not self.swap_node_x: self.swap_node_x = self.pre_node
				self.swap_node_y = root
		
			self.pre_node = root

			_inOrderTraversal(root.right)

		_inOrderTraversal(root)

		if self.swap_node_x and self.swap_node_y:
			self.swap_node_x.val, self.swap_node_y.val = self.swap_node_y.val, self.swap_node_x.val


n = 3
solution = Solution()
# solution.numTrees(n)
# ans = solution.generateTrees(n)
# for root in ans:
# 	print(convertBinaryTree2List(root))
root = [5,1,4,None,None,3,6]
root = [2,1,3]
root = [32,26,47,19,None,None,56,None,27]
# root = [5,4,6,None,None,3,7]
# root = [0, None, -1]


root = [1,3,None,None,2]
root = constructBinaryTree(root)
# solution.isValidBST2(root)
solution.recoverTree(root)
convertBinaryTree2List(root)

None
None
None
None


[3, 1, None, None, 2]

In [ ]:
class Solution:
	def isSameTree(self, p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
		if not p and not q: return True
		if not p or not q: return False

		if p.val != q.val:
			return False
			
		return self.isSameTree(p.left, q.left) and self.isSameTree(p.right, q.right)
	
	# recursive algorithm.
	def isSymmetric(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		def _isSymmetric(p: Optional[TreeNode], q: Optional[TreeNode]) -> bool:
			if not p and not q: return True
			if not p or not q: return False

			if p.val != q.val:
				return False
			return _isSymmetric(p.left, q.right) and _isSymmetric(p.right, q.left)

		return _isSymmetric(root.left, root.right)
	
	# level traversal.
	def isSymmetric_stack(self, root: Optional[TreeNode]) -> bool:
		if not root: return False

		queue = [root]

		while queue:
			n = len(queue)
			level_vals = []

			for _ in range(n):
				node = queue.pop(0)
				if node: 
					queue.append(node.left)
					queue.append(node.right)
					level_vals.append(node.val) 
				else: 
					level_vals.append(node)
			
			for i in range(n // 2 + 1):
				if level_vals[i] != level_vals[n-i-1]:
					return False

		return True


p = [1 ,2]
q = [1,None,2]

p = [1,2,3]
q = [1,2,3]

p = [1,2,1]
q = [1,1,2]
p, q = constructBinaryTree(p), constructBinaryTree(q)

solution = Solution()
ans = solution.isSameTree(p, q)
ans

root = [1,2,2,None,3,None,3]
# root = [1,2,2,3,4,4,3]
root = constructBinaryTree(root)
solution = Solution()
ans = solution.isSymmetric_stack(root)
ans

False

In [16]:
class Solution:
	def zigzagLevelOrder(self, root: Optional[TreeNode]) -> List[List[int]]:
		if not root: return []

		ans = []
		queue = [root]
		order = True

		while queue:
			n = len(queue)
			level_vals = []

			for _ in range(n):
				node = queue.pop(0)
				level_vals.append(node.val)
				
				if node.left: queue.append(node.left)
				if node.right: queue.append(node.right)
			if order:
				ans.append(level_vals)
			else:
				level_vals.reverse()
				ans.append(level_vals)
		
			order = not order
			
		return ans
	

root = [3,9,20,None,None,15,7]
root = constructBinaryTree(root)
solution = Solution()
solution.zigzagLevelOrder(root)

[[3], [20, 9], [15, 7]]

In [ ]:
# The max and min depth of a binary tree
class Solution:
	def maxDepth(self, root: Optional[TreeNode]) -> int:
		if not root: return 0

		depth = 0
		queue = [root]

		while queue:
			depth += 1
			
			for _ in range(len(queue)):
				node = queue.pop(0)
				if node.left: queue.append(node.left)
				if node.right: queue.append(node.right)
		
		return depth

	def minDepth(self, root: Optional[TreeNode]) -> int:
		if not root: return 0

		depth = 0
		queue = [root]

		while queue:
			depth += 1
			for _ in range(len(queue)):
				node = queue.pop(0)
				if not node.left and not node.right: return depth
				if node.left: queue.append(node.left)
				if node.right: queue.append(node.right)
        

root = [3,9,20,None,None,15,7]
root = constructBinaryTree(root)
solution = Solution()
solution.maxDepth(root)
solution.minDepth(root)

2